In [2]:
import h5py 
import numpy as np
import tiledb
import threading
import time
import threading

In [3]:
class TileDBSlicer:
    def __init__(self,blockOffset=256,cacheSize=0,filename="/scratch/aoyagir/tiledb_data/array64") -> None:
        # Create a TileDB config
        self.config = tiledb.Config()
        self.ctx = tiledb.Ctx(self.config)
        self.filename = filename
        self.blockOffset = blockOffset
        self.timesteps = None
        self.xMax = None
        self.yMax = None
        self.zMax = None
        self.getDim()
        self.printDataInfo()


    def printDataInfo(self):
        print("##### Row data INFO #####")
        print("timesteps:{}\nx:{}\ny:{}\nz:{}\n"
              .format(self.timesteps,self.xMax,self.yMax,self.zMax))

    def getDim(self):
        with tiledb.open(self.filename, mode="r", ctx=self.ctx) as array:
            # Retrieve and print the array schema
            array_schema = array.schema
            print("Array Schema:")
            print(array_schema)
            # You can also access other information about the array, such as its domain and attributes
            domain = array_schema.domain
            print("Domain:", domain)

    def get(self,timestep,x,xx,y,yy,z,zz):
        with tiledb.DenseArray(self.filename, mode="r", ctx=self.ctx) as array:
            # Read the sliced data from the TileDB array using 'from' and 'to' expressions
            # arrayから返却されるのは、orderd_dict.で、keyが'data'になっているので読みだす必要がある。
            # 以下で、dataが読み出される。
            data = array[timestep,x:xx, y:yy, z:zz]['data']
            return data



In [4]:

class Slicer:
    # default 引数
    def __init__(self,blockOffset=256,filename="/scratch/aoyagir/step1_256_test_0902.h5") -> None:
        self.filename = filename
        self.blockOffset = blockOffset
        self.file = h5py.File(filename, 'r')
        self.dataset = self.file['data']
        self.timesteps = self.dataset.shape[0]
        self.xMax = self.dataset.shape[1]
        self.yMax = self.dataset.shape[2]
        self.zMax = self.dataset.shape[3]
        self.lock = threading.Lock()
        self.printDataInfo()

    # Access specific elements in the concatenated array
    def access_array_element(self,timestep, x, y, z):
        element = self.dataset[timestep, x, y, z]
        return element

    # Access a subset of the concatenated array
    def slice_multiple_step(self, file, timestep_start, timestep_end, x_start, x_end, y_start, y_end, z_start, z_end):
        subset = self.dataset[timestep_start:timestep_end, x_start:x_end, y_start:y_end, z_start:z_end]
        return subset
    
    def slice_single_step(self, timestep,  x_start, x_end, y_start, y_end, z_start, z_end):
        subset = self.dataset[timestep,  x_start:x_end, y_start:y_end, z_start:z_end]
        retsubset = np.squeeze(subset)
        return retsubset
    
    # blockId = (tol,timestep,x,y,z)
    def sliceData(self,blockId):
        # with self.lock:
        _ = blockId[0]
        t = blockId[1]
        x = blockId[2]
        y = blockId[3]
        z = blockId[4]
        return self.slice_single_step(t,x,x+self.blockOffset,y,y+self.blockOffset,z,z+self.blockOffset)

    def changeBlockSize(self,blockSize):
        print("changin block size from {} to {}".format(self.blockOffset,blockSize))
        self.blockOffset = blockSize
    
    def printBlocksize(self):
        print(self.blockOffset)

    def printDataInfo(self):
        print("##### Row data INFO #####")
        print("timesteps:{}\nx:{}\ny:{}\nz:{}\n"
              .format(self.timesteps,self.xMax,self.yMax,self.zMax))
        
    def getDataDim(self):
        return self.dataset.shape

In [10]:
t_slicer = TileDBSlicer()
h_slicer = Slicer()

a = t_slicer.get(0,0,1024,0,1024,0,1024)
b = h_slicer.slice_single_step(0,0,1024,0,1024,0,1024)

Array Schema:
ArraySchema(
  domain=Domain(*[
    Dim(name='timestep', domain=(0, 63), tile=1, dtype='uint64', filters=FilterList([ZstdFilter(level=-1), ])),
    Dim(name='x', domain=(0, 1023), tile=256, dtype='uint64', filters=FilterList([ZstdFilter(level=-1), ])),
    Dim(name='y', domain=(0, 1023), tile=256, dtype='uint64', filters=FilterList([ZstdFilter(level=-1), ])),
    Dim(name='z', domain=(0, 1023), tile=256, dtype='uint64', filters=FilterList([ZstdFilter(level=-1), ])),
  ]),
  attrs=[
    Attr(name='data', dtype='float32', var=False, nullable=False, enum_label=None),
  ],
  cell_order='row-major',
  tile_order='row-major',
  sparse=False,
)

Domain: Domain(Dim(name='timestep', domain=(0, 63), tile=1, dtype='uint64', filters=FilterList([ZstdFilter(level=-1), ])),
       Dim(name='x', domain=(0, 1023), tile=256, dtype='uint64', filters=FilterList([ZstdFilter(level=-1), ])),
       Dim(name='y', domain=(0, 1023), tile=256, dtype='uint64', filters=FilterList([ZstdFilter(level=-1

In [11]:
a - b

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.